<a href="https://colab.research.google.com/github/5harad/DPI-617/blob/main/labs/included-variable-bias-answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Law, Order, and Algorithms**
## Included-variable bias and risk-adjusted regression

**Getting started**

Before you start, create a copy of this Jupyter notebook in your own Google Drive by clicking `Copy to Drive` in the menubar. If you do not do this your work will not be saved! 

Remember to save your work frequently by pressing command-S or clicking File > Save in the menubar. 

We recommend completing this problem set in Google Chrome.

Run the cell below to load the `tidyverse` library and set some formatting options.

In [ ]:
# Some initial setup
options(digits = 3)
library(tidyverse)

# Set some formatting options
options(digits = 3, repr.matrix.max.rows = 10, repr.matrix.max.cols = 100)
theme_set(theme_bw())

## Background
In this lab, we will investigate how to use a regression model to measure disparities across different groups, and discuss some of the problems that might arise in doing so. We will use the NYC stop and frisk data we have been using in previous labs.

Run the cell below to load and parse the data.

In [ ]:
# For computational reasons, we'll work with a sample of the data.
# We also exclude rare suspected crime categories, and relevel 
# the race variable so that "white" is the base category
fname <- "https://github.com/5harad/DPI-617/blob/main/data/sqf_sample.rds?raw=true"

set.seed(1)
stop_df <- readRDS(url(fname)) %>%
  sample_n(1e5) %>%
  group_by(suspected_crime) %>%
  filter(n() >= 10) %>%
  ungroup() %>%
  mutate(
    subject_race = as.factor(str_to_title(subject_race)),
    subject_race = relevel(subject_race, "White")
  )

head(stop_df)

## Base rate disparities in the decision to frisk

First, let's measure the disparities in police decisions to frisk individuals of different race groups.

### Exercise 1: raw disparities

For each race group, compute the proportion that were frisked

In [ ]:
# With the stop_df data, group by subject_race and then compute the 
# proportion of people who where frisked
# WRITE CODE HERE
# START solution
stop_df %>%
    group_by(subject_race) %>%
    summarize(p_frisked = mean(frisked))
# END solution

### Base rate disparities with regression

Another method for comparing differences in frisk rates is to use regression. 

In [ ]:
base_model <- lm(frisked ~ subject_race, data = stop_df)
summary(base_model)

The coefficients above give the increase in frisk rates for racial minorities, relative to stopped white pedestrians.

## Adjusting for other variables

One concern is that officers might have a "legitimate" reason to frisk certain individuals more often; it might just be that that reason is also highly correlated with race.

For example, as we discussed in earlier labs, one of the reasons for stopping an individual is if the officer suspects criminal posession of a weapon (encoded in the `suspected_crime` column as `cpw`).
Given that the primary justification of a frisk is concern for officer safety, one could argue that it is reasonable for an officer to 
frisk individuals whom they have stopped under suspicion of criminal posession of weapons.

(Although, whether an officer's _suspicion_ itself is justified is a different question, which we will address later)

### Adjusting for `suspected_crime == "cpw"` 

Given a regression model, we can "adjust for" additional variables in our data by including them in the right-hand side of our formula.

### Exercise 2: 

With `stop_df`, first create a new binary column named `is_cpw` that is `TRUE` if `suspected_crime` is `cpw`. Then fit a model that adjusts for this new covariate, and discuss the results.

In [ ]:
# WRITE CODE HERE
# START solution
stop_df <- stop_df %>%
  mutate(is_cpw = suspected_crime == "cpw")

cpw_model <- lm(frisked ~ subject_race + is_cpw, data = stop_df)
summary(cpw_model)
# END solution

## A kitchen-sink approach 

By the above logic, there might also be other "legitimate" factors that officers use in making frisk decisions. One common method for measuring disparities is to include _all_ recorded data that would have been available to the officer at the time of making the decision (to frisk an individual). We call this the "kitchen sink" approach.

The code below applies the kitchen sink approach to estimates difference in frisk rates across race groups after adjusting for everything else in our data.

In [ ]:
feats <- c(
    "subject_race",
    "suspected_crime",
    "precinct",
    "location_housing",
    "subject_sex",
    "subject_age",
    "subject_height",
    "subject_weight",
    "subject_build",
    "additional_report",
    "additional_investigation",
    "additional_proximity",
    "additional_evasive",
    "additional_associating",
    "additional_direction",
    "additional_highcrime",
    "additional_time",
    "additional_sights",
    "additional_other",
    "stopped_bc_object",
    "stopped_bc_desc",
    "stopped_bc_casing",
    "stopped_bc_lookout",
    "stopped_bc_clothing",
    "stopped_bc_drugs",
    "stopped_bc_furtive",
    "stopped_bc_violent",
    "stopped_bc_bulge",
    "stopped_bc_other"
)

# This creates a formula with a specified left-hand side (response = "frisked"),
# and using all the variables in feats on the right-hand side. 
# Constructing a formula in this way (instead of typing out all the variable names)
# is helpful for constructing multiple models that share a long list of variables in the right-hand side.
kitchen_sink_formula <- reformulate(feats, response = "frisked")

# We are only interested in the race coefficients
ks_model <- lm(kitchen_sink_formula, stop_df)
summary(ks_model)

### Exercise 3:

Do you think this is a reasonable approach to measuring disparate impact?
What about disparate treatment?

## Risk-adjusted regression

One problem with including all variables in measuring disparate impact is that an empirical connection between a factor and a decision is not necessarily justified. Blindly including a variable in a regression fails to take into account this _degree_ of justification, 
potentially overcompensating for variables that are correlated with race. This is the problem known as _included-variable bias_.

One simple idea for addressing this concern of included-variable bias is to control for an explicit measure of **risk**, instead of controling for individual variables.
Intuitively, we wish to know whether individuals who have _similar risk_ (of carrying a weapon) were treated (frisked) equally.

### Exercise 4: Estimating risk

In order to adjust for risk, we must first estimate it. This is relatively straightforward in the context of frisk decisions in stop-and-frisk, 
because the goal of a frisk is relatively clear -- we wish to recover weapons. 
In other words, we want to predict whether a weapon would be found if an individual is frisked. 

* **Step 1**: `filter` the `stop_df` data to those individuals _who were frisked_. We will call this new data frame `frisked_df`

The _risk_ that we are interested in estimating is the probability that a weapon is recovered given that we _frisk_ someone who has already been stopped.
While there are many ways to achieve this, one simple way is to build a predictive model, estimating the probability that a weapon is recovered, 
using just the data for stopped individuals who happened to be frisked. 
(Implicitly, this relies on an assumption of [_ignorability_](https://en.wikipedia.org/wiki/Ignorability).)

In [ ]:
# Subset the stop_df data to cases where the individual was frisked
# WRITE CODE HERE
# START solution
frisked_df <- stop_df %>%
    filter(frisked)
# END solution

* **Step 2**: Using the `frisked_df` data, we now fit a regression model to predict whether or not a weapon is found using all features that would reasonably be available to an officer (as listed in `feats` above). Let's call this model `risk_model`. 


In [ ]:
# Using the subset of data from Step 1, we fit the regression model: 
# found_weapon ~ (all available features in feats) 

risk_formula <- reformulate(feats, response = "found_weapon")
risk_model <- lm(risk_formula, data = frisked_df)

* **Step 3**: We use the `risk_model` from above to generate a column of model estimated risk (we'll name this column `risk`) on the original `stop_df` data. 

In [ ]:
# Generate a column of predicted risk 
stop_df <- stop_df %>%
    mutate(risk = predict(risk_model, .))

## Distribution of risk

Now that we have an estimate of risk, we can explore the distribution of risk across race groups.

In [ ]:
ggplot(filter(stop_df, subject_race %in% c('White','Black','Hispanic')), aes(x = risk)) +
geom_histogram(binwidth = .01) +
facet_wrap(~ subject_race) +
scale_x_continuous("Estimated probability of recovering weapon if frisked",
                   labels = scales::percent_format()) +
coord_cartesian(xlim = c(0, .1))

### Exercise 5: Risk-adjusted regression

Now compute _risk-adjusted_ frisk rates for stopped individuals across different race groups.

How do these results compare to both the raw base rates and the "kitchen sink" approach?

In [ ]:
# WRITE CODE HERE
# START solution
rar_model <- lm(frisked ~ subject_race + risk, stop_df)
summary(rar_model)
# END solution